# MUM 2023-24 Regularyzacja

## Problem i cel
* model jest budowany z wykorzystaniem dostępnych danych
  * dostępnych danych jest zwykle zbyt mało i budowa modelu z reguły prowadzi do jednego z dwóch problemów
    * __underfitting__ gdy model zbytnio uogólnia
    * __overfitting__ gdy model jest zbyt szczegółowy i zbyt dokładnie dopasowuje się do dostępnych danych [overfitting](./23_Overfitting.ipynb)
  * regularyzacja powinna poprawić __wynik na zbiorze testowym__, często kosztem danych treningowych

<img style="float: right;" src="ml_figures/function-space-reduction.png" width=450>  
    
* regularyzacja może być
  * dodaniem pewnej wiedzy wstępnej _a priori_ do modelu przez użycie założonego rozkładu parametrów
    * to ogranicza __pojemność modelu__ jednocześnie ograniczając overfitting
  * ograniczeniem przestrzeni funkcji, np. ograniczenie użytych wykładników w aproksymacji wielomianowej  * dowolną modyfikacją uczenia poprawiającą generalizację kosztem błędu uczenia (Ian Goodfellow)
* generalnie regularyzacja jest związana z ograniczeniem liczby parametrów
  * miary AIC (Akaike) i BIC (Bayesian) Information Criterion

#### Bardzo ważna uwaga
1. regularyzacja modyfikuje funkcję kosztu __tylko na zbiorze treningowym__
2. na __zbiorze testowym__ sprawdzamy __oryginalną funkcję kosztu__ (i ew. inne metryki, np. accuracy)
3. czy jest jasne, dlaczego należy tak robić?

## Najpopularniejsze modele regularyzacji
  * weight decay
  * early stopping
  * dropout
  * batch normalization

## Podejścia do uczenia
* podstawy do podejść do uczenia w notebooku [MLE i MAP](./stat_04_Twierdzenie_Bayesa.ipynb)

### Uczenie Maximum Likelihood Estimator (MLE)
* Szukamy takiego $w_\ast$, które zmaksymalizuje wartość liczbową likelihoodu:
$$w_\ast = \underset{w}{\arg\max}\;p\,(D_{Tr}\mid{}w)$$
  * gdzie $w$ jest zestawem parametrów odpowiadającym przyjętej hipotezie
  * MLE szuka najlepszych parametrów z obserwowanych przykładów
  * w MLE funkcja kosztu estymuje jak rozkład predykcji modelu odpowiada rozkładowi danych uczących
  * MLE jest **spójny**: wraz ze wzrostem liczby przykładów uczących poprawia się estymacja parametrów modelu


### Uczenie Maximum a Posteriori (MAP)
1. oznaczmy przez $w$ wektor wszystkich parametrów modelu
2. oznaczmy przez $L_{MLE}$ funkcję kosztu negative mean log likelihood (z dokładnością do stałych)
3. oznaczmy przez $L_{MAP}$ funkcję kosztu __negative mean log posterior__ (z dokładnością do stałych)
  * definiujemy podobnie do $L_{MLE}$
  * z tą różnicą, że zamiast likelihood weźmiemy posterior
4. obliczmy $L_{MAP}$ korzystając ze wzoru Bayesa
$$\begin{align}
p(w\mid D_{Tr}) &= \dfrac{p(D_{Tr}\mid{}w)p(w)}{p(D_{Tr})} \\
\ln[p(w\mid D_{Tr})] &= \ln[p(D_{Tr}\mid{}w)] + \ln[p(w)] - \overbrace{\ln[p(D_{Tr})]}^{const} \\
-\dfrac{1}{N}\ln[p(w\mid D_{Tr})] &= -\dfrac{1}{N}\ln[p(D_{Tr}\mid{}w)] -\dfrac{1}{N}\ln[p(w)] + const \\
L_{MAP}(w) &= L_{MLE}(w) -\dfrac{1}{N}\ln[p(w)]
\end{align}$$
  * w definicji pomijamy $const$
    * zależny od liczby przykładów $N$ oraz od $p$ i niezależny od wartości $w$
    * można pominąć, bo maksymalizujemy po zmiennej $w$
5. prosta reguła na regularyzację metodą MAP
  * weź funkcję kosztu MLE
  * dodaj składnik $-\dfrac{1}{N}\ln[p(w)]$
    * __zależny od parametrów modelu__ $w$
    * __niezależny od danych treningowych__

#### Uwaga
1. $L_{MLE}$ to __średnia__ po $N$ przykładach
2. w takim razie $\ln[p(\theta)]$ musi być podzielone przez $N$
  * jeśli $N$ dąży do nieskończoności, to wpływ wiedzy a priori musi dążyć do zera
3. niekiedy w literaturze $\ln[p(\theta)]$ nie jest dzielone przez $N$, ale wtedy funkcja kosztu to negative log likelihood, a nie negative __mean__ log likelihood
  * oba przypadki są równoważne

### Jak wybrać prior
* mamy pełną dowolność w zdefiniowaniu $p(w)$
  * ale tylko "dobry" prior pomoże w uczeniu
  * np. trzeba uważać, aby prior nie był zero dla żadnej sensownej hipotezy
* dobrze zdefiniować go tak, aby łatwo liczył się logarytm
  * funkcja $\exp$ w definicji gęstości — wtedy się wykasuje z logarytmem
  * współrzędne $w$ to zmienne niezależne — wtedy $\ln[p(w)] = \sum_{t=1}^T\ln[p(w)]$
* ważne są problemy dokładności i stabilności obliczeń

## Złożoność modelu a jego generalizacja
<img style="float: left;" src="ml_figures/error-complexity.png" width=440>

* generalizacja opisuje skuteczność modelu na przykładach spoza zbioru uczącego $$Err_{tst}=E[L(Y, f(x))\mid D_{tst}]$$
* $err$ to średni loss na zbiorze uczącym $D_{Tr}$
  * błąd na zbiorze uczącym niekoniecznie jest dobrym estymatorem generalizacji
* rozkład błędu modelu $\hat{f}$ na _bias_ i _wariancję_
  * niech szumem modelu będzie $\epsilon\sim\mathcal{N}(0,\sigma)$
$$\begin{align*}
Err(\hat{f})&=E[(y-\hat{f})^2]&=E[y^2]-2E[y\hat{f}]+E[\hat{f}^2]\\
\\
&\textrm{gdzie poszczególne elementy}\\
E[y^2]&=E[(f+\epsilon)^2]=E[f^2]+2E[f\epsilon]+\epsilon^2\\
&=E[f^2]+2E[f]\cdot0+\epsilon^2&=f^2+\sigma^2\\
\\
&\textrm{dla każdej zmiennej losowej zachodzi}\\
Var(X)&=E[(X-E[X])^2]=E[X^2]-E[X]^2\\
&\textrm{stąd}\\
E[\hat{f}^2]&=&=Var(\hat{f})+E[\hat{f}]^2\\
\\
E[y\hat{f}]&=E[(f+\epsilon)\hat{f}]\\
&=E[f\hat{f}]+E[\epsilon\hat{f}]&=fE[\hat{f}]\\
\\
&\textrm{podstawiając}\\
Err[\hat{f}]&=f^2+\sigma^2-2E[\hat{f}]+Var(\hat{f})+E[\hat{f}]^2\\
&=f^2-2f{}E[\hat{f}]+E[\hat{f}]^2+\sigma^2+Var(\hat{f})\\
&=(f-E[\hat{f}])^2+\sigma^2+Var(\hat{f})&=Bias[\hat{f}]^2+\sigma^2+Var(\hat{f})
\end{align*}$$
  <img style="float: right;" src="ml_figures/bias-variance-triangle.png" width=400>

  * $\sigma^2$ jest nieredukowalnym składnikiem błędu, niemożliwym do usunięcia
  * $Bias[\hat{f}]^2$ określa o ile średnia przybliżeń różni się od prawdziwej średniej
  * $Var(\hat{f})$ jest wariancją modelu $\hat{f}$, oczekiwaną zmiennością funkcji wokół jej średniej

## Regularyzacja L2 (weight decay)
1. niech priorem będzie rozkład normalny
  * zakładamy, że $w_t\sim\mathcal{N}(0,\sigma^2)=\dfrac{1}{\sigma\sqrt{2\pi}}\exp\left(-\dfrac{1}{2}\left(\dfrac{w_t}{\sigma}\right)^2\right)$ dla $t\in\{1,\ldots,T\}$
  * współrzędne $w$ są a priori niezależne
$$\begin{align}
\ln[p(w)] &= \sum_{t=1}^T\ln[p(w_t)] \\
&= \sum_{t=1}^T \ln[1/\sigma] + \ln[1/\sqrt{2\pi}] - \dfrac{w_t^2}{2\sigma^2} \\
&= const - \lambda \sum_{t=1}^T w_t^2 \\
&= const - \lambda \|w\|^2 \\
L_{MAP}(w) &= L_{MLE}(w) + \dfrac{\lambda}{N}\|w\|^2 
\end{align}$$
2. $const$ zależy tylko od $T$ i $\sigma$, możemy pominąć przy optymalizacji gradientowej $w$
3. parametr $\lambda$ może być dowolną dodatnią liczbą rzeczywistą
  * możemy wybrać dowolną liczbę, ponieważ możemy wybrać dowolne $\sigma$
    * czy widać, w jaki sposób $\lambda$ zależy od $\sigma$?
    * co się stanie, jeśli wybierzemy ujemne $\lambda$?
  * gdy $\lambda\to 0$, to $L_{MAP}\to L_{MLE}$
    * jak zmienia się wtedy prior? jaka jest jego "granica"?
    * czy to jest zgodne z naszymi oczekiwaniami? 
4. efekt regularyzacji L2
  * ograniczenie sumy __kwadratów__ współrzędnych
  * duże (na moduł) wartości $\theta_t$ są bardzo kosztowne
    * stąd nazwa __weight decay__
      * uwaga na terminologię - w literaturze "weight decay coefficient" może oznaczać $\lambda$ lub $2\lambda$
  * jeśli działanie fragmentu modelu zależy (w przybliżeniu) od __sumy__ $M$ współrzędnych $w_{t_1}, \ldots, w_{t_M}$, to model nauczy się tam (w przybliżeniu) równych wartości liczbowych
5. Weight decay wpływa na _efektywną_ liczbę parametrów
  * parametry nieistotne dla modelu powinny być redukowane do zera
    * L2 (sfera na układzie współrzędnych wag) redukuje pojedyncze wagi bardziej równomiernie
    * L1 (wielościan z rogami na wagach) robi to bardziej nierównomiernie redukując bardziej nieistotne wagi bardziej
      * jest 
  * efektyna liczba parametrów jest zwykle określona przez $$df(\lambda)=\sum_k\dfrac{\theta_k}{\theta_k+\lambda}$$ gdzie $\theta_k$ to wartości własne macierzy Hesjanu funkcji kosztu $\dfrac{\nabla^2{}L(w)}{\nabla{}w\nabla{}w^T}$, a więc duże $\lambda$ redukuje bardziej (jak można się było spodziewać)
6. regresja liniowa z regularyzacją L2 to tzw. __ridge regression__

## Inne popularne rodzaje regularyzacji typu MAP

1. __regularyzacja L1__ (__lasso__)
$$L_{MAP}(w) = L_{MLE}(w) + \dfrac{\lambda}{N}\|w\|$$
  * norma L1 zamiast normy L2
  * odpowiada innemu priorowi
    * podobnie jak w przypadku regresji liniowej - MSE vs MAE
  * jeśli działanie fragmentu modelu zależy w przybliżeniu od sumy $M$ współrzędnych $w_{t_1}, \ldots, w_{t_M}$, to modelowi opłaca się ustawić na 0 wszystkie $M-1$ współrzędnych poza jedną najistotniejszą
    * regularyzacja L1 zachęca do uczenia __rzadkiego__ (_sparse_) wektora $w$
    * zwiększa __interpretowalność__ - łatwiej przeanalizować kilka najistotniejszych współrzędnych

2. połączenie L1 i L2 (__elastic net__)
$$L_{MAP}(w) = L_{MLE}(w) + \dfrac{\lambda_1}{N}\|w\| + \dfrac{\lambda_2}{N}\|w\|^2$$

## Regularyzacja w praktyce

1. zazwyczaj wystarczy użyć L2
2. należy przetestować wiele różnych $\lambda$ (nauczyć kilka modeli)
3. można dobierać różne $\lambda$ dla różnych współrzędnych
  * np. różne $\lambda$ dla różnych warstw sieci neuronowej
  * ale jeśli to nie ma dobrego uzasadnienia, to zazwyczaj szkoda na to czasu
4. powyższe regularyzacje zbliżają $w$ do zera
  * np. sieciach neuronowych ustawienie wszystkich wag na 0 z reguły sprawi, że klasyfikator zwróci rozkład jednostajny, a regressor zero 
  * można oczywiście zrobić inaczej, np. użyć $\sum_{t=1}^T (w_t - 1)^2$
    * ale musimy mieć dobre uzasadnienie, że wszystkie współrzędne powinny wynosić w przybliżeniu 1
    * zazwyczaj nie mamy
5. można definiować własne regularyzacje
  * pomysłowe regularyzacje mogą znacznie poprawić uczenie modelu
  * nie trzeba przejmować się, czy odpowiadają jakiemukolwiek priorowi
    * w najgorszym razie model nie zadziała, ale warto dlaczego
    <!-- * warto rozumieć związek regularyzacji L2 oraz L1 z priorem ;-) -->

### Niezmienniki (inwariancje)
* jeśli dla wielu wzorów predykcje nie mają się zmieniać, to odpowiada _wbudowywaniu niezmienników_ (_inwariancji_)
  * szczególnie w rozpoznawaniu wzorców (translacje, obroty), czy dźwięków (np. głośność)
  * przy odpowiedniej liczbie wzorców model może nauczyć się niezmienników
    * można dodać odpowiednią liczbę powtórzeń przykładów odpowiednio zmodyfikowanych (augmentacje)
    * dzielone wagi oraz tzw. pooling, jak w przypadku sieci konwolucyjnych, mogą być przykładem wbudowanych do architektury sieci niezmienników
* niezmienniki są modelem wykorzystywanym w tzw. uczeniu ustawicznym (ciągłym, ang. continual learning)
  * model uczy się szeregu następujących po sobie problemów
  * po nauczeniu problemu $X_k$, pojawia się kolejny $X_{k+1}$
  * algorytm uczący **nie ma już dostępu** do danych $X_k$
  * wprowadzenie niezmienników odpowiada temu, by predykcje dla przykładów z $X_k, X_{k-1},\dots$ pozostały bez zmian
  * to jest bardzo trudne zadanie, ale aktualne
* wiele potrzebnych augmentacji dla standardowych problemów jest dostepnych w bibliotekach, np. `torchvision.transforms`

## Early stopping
* podobnie jak w przypadku zależności błędu uczącego i generalizacji w zależności od złożoności modelu, jest on zależny od długości uczenia (liczby epok)
  * model wraz z liczbą epok staje się coraz bardziej 'dopasowany', a stąd i złożony
  * pewnym rozwiązaniem może być wcześniejsze zatrzymanie uczenia
  * wymaga wielu prób i zapamiętywania modeli